In [1]:
import numpy as np
import dill
import pandas as pd

import datetime as dt
from tqdm import tqdm
from collections import defaultdict, Counter

import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")

## Diagnosis

In [5]:
ICD_9 = ['V462', 'V1254', 'E8497', 'E8798', 'V440', 'V4611', 'V550', 'V5332', '9971', 'E8782', '99731', '24900', 'E8788', '99702', 'V1251', 'E8889', '9972', '99749', '99739', 'V5331', '9975', '43321', '43311', '99791', 'V420', '40491', '99701', '99709', '99769', '99762', '99779', '43331', 'V427', '39891', '40493', '43301']
ICD_10 = ['I95', 'E87', 'R10', 'F10', 'R68', 'J44', 'K70', 'J69', 'I50', 'J98', 'N17', 'K92', 'R09', 'I47', 'D64', 'I25', 'M15', 'M81', 'F41', 'I10', 'I34', 'I21', 'N18', 'I12', 'E11', 'Z51', 'K22', 'G93', 'J45', 'F32', 'Z91', 'I20', 'G61', 'E78', 'E66', 'Z95', 'J18', 'J41', 'R69', 'R00', 'R19', 'D50', 'A04', 'E89', 'A40', 'N39', 'T78', 'K56', 'J47', 'K57', 'R78', 'K76', 'D69', 'I42', 'I27', 'B15', 'J17', 'T81', 'G47', 'E44', 'F05', 'M10', 'L89', 'K26', 'I44', 'K31', 'J15', 'I24', 'Z85', 'I66', 'G44', 'K75', 'I85', 'I61', 'K85', 'D66', 'R93', 'N23', 'K25', 'D70', 'J93', 'H35', 'J30', 'N40', 'G40', 'R06', 'I73', 'F04', 'J81', 'K62', 'R20', 'I26', 'R45', 'T88', 'J84', 'I60', 'B25', 'J43', 'J13', 'T82', 'E88', 'I62', 'R94', 'G97', 'J86', 'J34', 'J95', 'E84', 'L94', 'I76', 'I67', 'I11', 'J40', 'Z23', 'K90', 'J85', 'B37', 'K28', 'J12']

In [7]:
D_ICD_DIAGNOSES = pd.read_csv('input/D_ICD_DIAGNOSES.csv.gz')
D_ICD_DIAGNOSES.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']

diag_9_10 = pd.read_csv('input/0_diag_9_10.csv')
diag_9_10.columns = ['ICD_CODE', 'ROOT']

In [ ]:
# MIMIC IV
IV_diagnoses_raw = pd.read_csv('input/diagnoses_icd.csv.gz')

# Filter diagnoses by ICD version (9 or 10) and relevant codes
IV_diagnoses_raw = IV_diagnoses_raw[
    (IV_diagnoses_raw.icd_code.isin(ICD_9) & (IV_diagnoses_raw.icd_version == 9)) |
    (IV_diagnoses_raw.icd_code.isin(ICD_10) & (IV_diagnoses_raw.icd_version == 10))
]

# Print shape and unique ICD codes for initial inspection
print(IV_diagnoses_raw.shape, len(IV_diagnoses_raw.icd_code.unique()))

# Select relevant columns and rename them to uppercase
IV_diagnoses_raw = IV_diagnoses_raw[['subject_id', 'hadm_id', 'icd_code', 'icd_version']]
IV_diagnoses_raw.columns = IV_diagnoses_raw.columns.upper()

# Group by subject and admission ID, creating a list of ICD codes per root category
IV_diagnoses_result = IV_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID', 'ROOT'])['ICD_CODE'].apply(list).reset_index()

# Pivot the grouped dataframe on HADM_ID and ROOT
IV_diagnoses_result = IV_diagnoses_result.pivot(index='HADM_ID', columns='ROOT', values='ICD_CODE')
IV_diagnoses_result = IV_diagnoses_result.reset_index()

# Merge raw diagnoses with additional diagnostic data (for ICD-9 and ICD-10)
IV_diagnoses = pd.merge(IV_diagnoses_raw[IV_diagnoses_raw.ICD_VERSION == 9], diag_9_10, on='ICD_CODE', how='left')
IV_diagnoses = pd.concat([IV_diagnoses, IV_diagnoses_raw[IV_diagnoses_raw.ICD_VERSION == 10]])

# Print updated shape and unique ICD codes
print(IV_diagnoses.shape, len(IV_diagnoses.ICD_CODE.unique()))

# Fill missing ROOT values with ICD_CODE and remove duplicates
IV_diagnoses['ROOT'] = IV_diagnoses['ROOT'].fillna(IV_diagnoses['ICD_CODE'])
IV_diagnoses = IV_diagnoses.drop_duplicates(subset=['HADM_ID', 'ICD_CODE', 'ICD_VERSION', 'ROOT'], keep='first')

# Print updated statistics
print(IV_diagnoses.shape, len(IV_diagnoses.ICD_CODE.unique()), len(IV_diagnoses.ROOT.unique()))
print(list(IV_diagnoses.ROOT.unique()))

# Print ICD version distribution
print('\n\n', IV_diagnoses.ICD_VERSION.value_counts())

# Merge ICD texts for ICD-9 and ICD-10 versions
IV_diagnoses_9 = pd.merge(
    IV_diagnoses[IV_diagnoses.ICD_VERSION == 9],
    D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 9][['ICD_CODE', 'ICD_TEXT']],
    on='ICD_CODE', how='left'
)

IV_diagnoses_10 = pd.merge(
    IV_diagnoses[IV_diagnoses.ICD_VERSION == 10],
    D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 10][['ICD_CODE', 'ICD_TEXT']],
    on='ICD_CODE', how='left'
)

# Concatenate both ICD-9 and ICD-10 diagnosis data
IV_diagnoses = pd.concat([IV_diagnoses_9, IV_diagnoses_10])

# Print final shape and unique ICD codes/roots
print(IV_diagnoses.shape, len(IV_diagnoses.ICD_CODE.unique()), len(IV_diagnoses.ROOT.unique()))

# Merge to get unique ICD_CODEs and ICD_TEXT for each SUBJECT_ID and HADM_ID
IV_diagnoses_merges00 = IV_diagnoses.groupby(by=['SUBJECT_ID', 'HADM_ID'])['ICD_CODE'].unique().reset_index()
IV_diagnoses_merges0 = IV_diagnoses_merges00.merge(
    IV_diagnoses.groupby(by=['SUBJECT_ID', 'HADM_ID'])['ICD_TEXT'].unique().reset_index(),
    on=['SUBJECT_ID', 'HADM_ID'], how='inner'
)
# Map ICD_TEXT to a list format
IV_diagnoses_merges0['ICD9_TEXT'] = IV_diagnoses_merges0['ICD_TEXT'].map(lambda x: list(x))

IV_diagnoses_result = IV_diagnoses.groupby(['SUBJECT_ID','HADM_ID', 'ROOT'])['ICD_CODE'].apply(list).reset_index()
IV_diagnoses_result = IV_diagnoses_result.pivot(index='HADM_ID', columns='ROOT', values='ICD_CODE')
IV_diagnoses_result = IV_diagnoses_result.reset_index()
IV_diagnoses_result.head()

In [ ]:
# MIMIC-III
III_diagnoses_raw = pd.read_csv('input/DIAGNOSES_ICD.csv.gz')

# Set ICD version to 9 for MIMIC-III data
III_diagnoses_raw['ICD_VERSION'] = 9

# Filter diagnoses based on ICD9 codes
III_diagnoses_raw = III_diagnoses_raw[III_diagnoses_raw.ICD9_CODE.isin(old_d)]

# Print shape and unique ICD9 codes for inspection
print(III_diagnoses_raw.shape, len(III_diagnoses_raw.ICD9_CODE.unique()))

# Select relevant columns and rename them to uppercase
III_diagnoses_raw = III_diagnoses_raw[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'ICD_VERSION']]
III_diagnoses_raw.columns = III_diagnoses_raw.columns.upper()

# Merge raw diagnoses data with the ICD-9 to ICD-10 conversion table
III_diagnoses = pd.merge(
    III_diagnoses_raw[III_diagnoses_raw.ICD_VERSION == 9],
    diag_9_10,
    on='ICD_CODE',
    how='left'
)

# Print updated shape and unique ICD codes
print(III_diagnoses.shape, len(III_diagnoses.ICD_CODE.unique()))

# Fill missing ROOT values with ICD_CODE and remove duplicates
III_diagnoses['ROOT'] = III_diagnoses['ROOT'].fillna(III_diagnoses['ICD_CODE'])
III_diagnoses = III_diagnoses.drop_duplicates(subset=['HADM_ID', 'ICD_CODE', 'ICD_VERSION', 'ROOT'], keep='first')

# Print updated shape, unique ICD codes, and ROOT values
print(III_diagnoses.shape, len(III_diagnoses.ICD_CODE.unique()), len(III_diagnoses.ROOT.unique()))
print(list(III_diagnoses.ROOT.unique()))

# Print the distribution of ICD versions
print('\n\n', III_diagnoses.ICD_VERSION.value_counts())

# Merge with ICD text descriptions for ICD-9 codes
III_diagnoses = pd.merge(
    III_diagnoses,
    D_ICD_DIAGNOSES[D_ICD_DIAGNOSES.ICD_VERSION == 9][['ICD_CODE', 'ICD_TEXT']],
    on='ICD_CODE',
    how='left'
)

# Group by subject and admission ID to get unique ICD codes and texts
III_diagnoses_merges00 = III_diagnoses.groupby(by=['SUBJECT_ID', 'HADM_ID'])['ICD_CODE'].unique().reset_index()
III_diagnoses_merges0 = III_diagnoses_merges00.merge(
    III_diagnoses.groupby(by=['SUBJECT_ID', 'HADM_ID'])['ICD_TEXT'].unique().reset_index(),
    on=['SUBJECT_ID', 'HADM_ID'],
    how='inner'
)

# Convert ICD_TEXT to a list format
III_diagnoses_merges0['ICD9_TEXT'] = III_diagnoses_merges0['ICD_TEXT'].map(lambda x: list(x))

# Display the first 2 rows of merged data
III_diagnoses_merges0.head(2)

# Group by subject and admission ID to list ICD codes per ROOT category
III_diagnoses_result = III_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID', 'ROOT'])['ICD_CODE'].apply(list).reset_index()
# Pivot the grouped data to create a matrix for HADM_ID and ROOT categories
III_diagnoses_result = III_diagnoses_result.pivot(index='HADM_ID', columns='ROOT', values='ICD_CODE')
# Reset the index after pivot
III_diagnoses_result = III_diagnoses_result.reset_index()
# Display the first few rows of the final result
III_diagnoses_result.head()

In [ ]:

union = list(set(III_diagnoses_result.columns)&set(IV_diagnoses_result.columns) - set(['HADM_ID']))
union = list(pd.unique(union))
print(len(union),union)

In [ ]:
III_diagnoses_result = III_diagnoses_result[['HADM_ID']+union]
IV_diagnoses_result = IV_diagnoses_result[['HADM_ID']+union]

print(III_diagnoses_result.shape,len(III_diagnoses_result.HADM_ID.unique()))
print(IV_diagnoses_result.shape,len(IV_diagnoses_result.HADM_ID.unique()))

In [ ]:
III_diagnoses_result_missing = III_diagnoses_result[union].isna().mean() * 100
IV_diagnoses_result_missing = IV_diagnoses_result[union].isna().mean() * 100

keep_d_ids = new_d + list(III_diagnoses_result_missing[III_diagnoses_result_missing<95].index) + list(IV_diagnoses_result_missing[IV_diagnoses_result_missing<95].index)
keep_d_ids = list(pd.unique(keep_d_ids))
print(len(keep_d_ids),keep_d_ids)

In [ ]:
III_diagnoses_result = III_diagnoses_result[['HADM_ID']+keep_d_ids]
IV_diagnoses_result = IV_diagnoses_result[['HADM_ID']+keep_d_ids]

print(III_diagnoses_result.shape,len(III_diagnoses_result.HADM_ID.unique()))
print(IV_diagnoses_result.shape,len(IV_diagnoses_result.HADM_ID.unique()))

In [ ]:
III_diagnoses_result.loc[:, III_diagnoses_result.columns != 'HADM_ID'] = III_diagnoses_result.loc[:, III_diagnoses_result.columns != 'HADM_ID'].notna().astype(int)
IV_diagnoses_result.loc[:, IV_diagnoses_result.columns != 'HADM_ID'] = IV_diagnoses_result.loc[:, IV_diagnoses_result.columns != 'HADM_ID'].notna().astype(int)

In [ ]:
III_diagnoses_result.to_csv('output/III_D.csv',index=False)
IV_diagnoses_result.to_csv('output/IV_D.csv',index=False)

## Procedures

In [ ]:
d_icd_procedures = pd.read_csv('input/d_icd_procedures.csv.gz')
d_icd_procedures.columns = ['ICD_CODE', 'ICD_VERSION', 'ICD_TEXT']
print(d_icd_procedures.ICD_VERSION.value_counts())
d_icd_procedures.head(2)

In [ ]:
IV_p_raw = pd.read_csv('input/procedures_icd.csv.gz')
IV_p_raw = IV_p_raw[['subject_id', 'hadm_id','icd_code','icd_version']]
IV_p_raw.columns = ['SUBJECT_ID', 'HADM_ID','ICD_CODE','ICD_VERSION']
IV_p_raw = IV_p_raw[IV_p_raw.ICD_VERSION == 9]
IV_p_raw['ICD_CODE'] = IV_p_raw['ICD_CODE'].astype(int)
IV_p_raw = IV_p_raw.drop_duplicates(keep='first')
print('IV_p_raw',IV_p_raw.shape,len(IV_p_raw.HADM_ID.unique()))

III_p_raw = pd.read_csv('input/PROCEDURES_ICD.csv.gz')
III_p_raw = III_p_raw[['SUBJECT_ID', 'HADM_ID','ICD9_CODE']]
III_p_raw.columns = ['SUBJECT_ID', 'HADM_ID','ICD_CODE']
III_p_raw['ICD_VERSION'] = 9
III_p_raw = III_p_raw.drop_duplicates(keep='first')
print('III_p_raw',III_p_raw.shape,len(III_p_raw.HADM_ID.unique()))

In [ ]:
union = list(set(III_p_raw.ICD_CODE.unique())&set(IV_p_raw.ICD_CODE.unique()))
union = list(pd.unique(union))
print(len(union))

In [ ]:
III_p_raw_result = III_p_raw[III_p_raw.ICD_CODE.isin(union)][['HADM_ID','ICD_CODE']].pivot_table(index='HADM_ID', columns='ICD_CODE', aggfunc='size', fill_value=np.nan)
III_p_raw_result = III_p_raw_result.reset_index()

IV_p_raw_result = IV_p_raw[IV_p_raw.ICD_CODE.isin(union)][['HADM_ID','ICD_CODE']].pivot_table(index='HADM_ID', columns='ICD_CODE', aggfunc='size', fill_value=np.nan)
IV_p_raw_result = IV_p_raw_result.reset_index()

In [ ]:
III_p_raw_result_missing = III_p_raw_result[union].isna().mean() * 100
IV_p_raw_result_missing = IV_p_raw_result[union].isna().mean() * 100

keep_p_ids = list(III_p_raw_result_missing[III_p_raw_result_missing<95].index) + list(IV_p_raw_result_missing[IV_p_raw_result_missing<95].index)
keep_p_ids = list(pd.unique(keep_p_ids))
print(len(keep_p_ids),keep_p_ids)

In [ ]:
III_p_raw_result = III_p_raw_result[['HADM_ID']+keep_p_ids]
IV_p_raw_result = IV_p_raw_result[['HADM_ID']+keep_p_ids]

print(III_p_raw_result.shape,len(III_p_raw_result.HADM_ID.unique()))
print(IV_p_raw_result.shape,len(IV_p_raw_result.HADM_ID.unique()))

In [ ]:
III_p_raw_result.loc[:, III_p_raw_result.columns != 'HADM_ID'] = III_p_raw_result.loc[:, III_p_raw_result.columns != 'HADM_ID'].notna().astype(int)
IV_p_raw_result.loc[:, IV_p_raw_result.columns != 'HADM_ID'] = IV_p_raw_result.loc[:, IV_p_raw_result.columns != 'HADM_ID'].notna().astype(int)

In [ ]:
III_p_raw_result.to_csv('output/III_P.csv',index=False)
IV_p_raw_result.to_csv('output/IV_P.csv',index=False)

## Drugs

In [ ]:
III = pd.read_csv('output/III.csv')
III_PRESCRIPTIONS = pd.read_csv('input/PRESCRIPTIONS.csv.gz', dtype={'NDC':'category'})
III_PRESCRIPTIONS = III_PRESCRIPTIONS[III_PRESCRIPTIONS.HADM_ID.isin(III.HADM_ID)]

III_PRESCRIPTIONS.STARTDATE = pd.to_datetime(III_PRESCRIPTIONS.STARTDATE)
III_PRESCRIPTIONS.ENDDATE = pd.to_datetime(III_PRESCRIPTIONS.ENDDATE)

In [ ]:
III_PRESCRIPTIONS = III_PRESCRIPTIONS[III_PRESCRIPTIONS.ICUSTAY_ID.isin(III.ICUSTAY_ID)]

III_PRESCRIPTIONS = pd.merge(III_PRESCRIPTIONS, III.drop(['HADM_ID'], axis=1), on='ICUSTAY_ID', how='left')
III_PRESCRIPTIONS['STARTDATE_IN_RANGE'] = (III_PRESCRIPTIONS['INTIME'] <= III_PRESCRIPTIONS['STARTDATE']) & (III_PRESCRIPTIONS['STARTDATE'] <= III_PRESCRIPTIONS['OUTTIME'])
III_PRESCRIPTIONS['STARTDATE_afterICU'] = (III_PRESCRIPTIONS['OUTTIME'] <= III_PRESCRIPTIONS['STARTDATE']) & (III_PRESCRIPTIONS['STARTDATE'] <= III_PRESCRIPTIONS['DISCHTIME'])

III_PRESCRIPTIONS = III_PRESCRIPTIONS[III_PRESCRIPTIONS.STARTDATE_afterICU==True]
III_PRESCRIPTIONS = III_PRESCRIPTIONS[['HADM_ID','ICUSTAY_ID','DRUG','NDC']]
III_PRESCRIPTIONS.shape

In [ ]:
# Step 1: 检查每个 DRUG 是否有非空的 NDC，如果没有，删除这些 DRUG 对应的所有行
drugs_with_non_empty_ndc = III_PRESCRIPTIONS.dropna(subset=['NDC']).groupby('DRUG').filter(lambda x: x['NDC'].notna().any())
III_PRESCRIPTIONS = III_PRESCRIPTIONS[III_PRESCRIPTIONS['DRUG'].isin(drugs_with_non_empty_ndc['DRUG'])]

# Step 2: 按照 DRUG 分组，计算每个 DRUG 对应的最常见的 NDC
most_common_ndc = III_PRESCRIPTIONS.dropna(subset=['NDC']).groupby('DRUG')['NDC'].agg(lambda x: x.mode()[0])

# Step 3: 用最常见的 NDC 填补缺失值
III_PRESCRIPTIONS['NDC'] = III_PRESCRIPTIONS.apply(lambda row: most_common_ndc[row['DRUG']] if pd.isna(row['NDC']) else row['NDC'], axis=1)

III_PRESCRIPTIONS = III_PRESCRIPTIONS.reset_index(drop=True)
III_PRESCRIPTIONS = III_PRESCRIPTIONS[~(III_PRESCRIPTIONS.NDC=='0')]
III_PRESCRIPTIONS = III_PRESCRIPTIONS.drop_duplicates(keep='first')
III_PRESCRIPTIONS = III_PRESCRIPTIONS.reset_index(drop=True)
print(III_PRESCRIPTIONS.shape,len(III_PRESCRIPTIONS.HADM_ID.unique()),len(III_PRESCRIPTIONS.ICUSTAY_ID.unique()))

In [ ]:
IV = pd.read_csv('output/IV.csv')
IV_PRESCRIPTIONS = pd.read_csv('input/prescriptions.csv.gz', dtype={'ndc':'category'})
IV_PRESCRIPTIONS.columns = IV_PRESCRIPTIONS.columns.str.upper()
IV_PRESCRIPTIONS = IV_PRESCRIPTIONS[IV_PRESCRIPTIONS.HADM_ID.isin(IV.HADM_ID)]

IV_PRESCRIPTIONS.STARTTIME = pd.to_datetime(IV_PRESCRIPTIONS.STARTTIME).dt.date
IV_PRESCRIPTIONS.STOPTIME = pd.to_datetime(IV_PRESCRIPTIONS.STOPTIME).dt.date

IV_PRESCRIPTIONS_isna = pd.merge(IV_PRESCRIPTIONS, IV, on='HADM_ID', how='left')

IV_PRESCRIPTIONS_isna['STARTDATE_IN_RANGE'] = (IV_PRESCRIPTIONS_isna['INTIME'] <= IV_PRESCRIPTIONS_isna['STARTTIME']) & (IV_PRESCRIPTIONS_isna['STARTTIME'] <= IV_PRESCRIPTIONS_isna['OUTTIME'])
IV_PRESCRIPTIONS_isna['STARTDATE_afterICU'] = (IV_PRESCRIPTIONS_isna['OUTTIME'] <= IV_PRESCRIPTIONS_isna['STARTTIME']) & (IV_PRESCRIPTIONS_isna['STARTTIME'] <= IV_PRESCRIPTIONS_isna['DISCHTIME'])

IV_PRESCRIPTIONS = IV_PRESCRIPTIONS_isna[IV_PRESCRIPTIONS_isna.STARTDATE_afterICU==True]
IV_PRESCRIPTIONS = IV_PRESCRIPTIONS[['HADM_ID','ICUSTAY_ID','DRUG','NDC']]
print(IV_PRESCRIPTIONS.shape,len(IV_PRESCRIPTIONS.HADM_ID.unique()),len(IV_PRESCRIPTIONS.ICUSTAY_ID.unique()))

In [ ]:
# Step 1: 检查每个 DRUG 是否有非空的 NDC，如果没有，删除这些 DRUG 对应的所有行
drugs_with_non_empty_ndc = IV_PRESCRIPTIONS.dropna(subset=['NDC']).groupby('DRUG').filter(lambda x: x['NDC'].notna().any())
IV_PRESCRIPTIONS = IV_PRESCRIPTIONS[IV_PRESCRIPTIONS['DRUG'].isin(drugs_with_non_empty_ndc['DRUG'])]

# Step 2: 按照 DRUG 分组，计算每个 DRUG 对应的最常见的 NDC
most_common_ndc = IV_PRESCRIPTIONS.dropna(subset=['NDC']).groupby('DRUG')['NDC'].agg(lambda x: x.mode()[0])

# Step 3: 用最常见的 NDC 填补缺失值
IV_PRESCRIPTIONS['NDC'] = IV_PRESCRIPTIONS.apply(lambda row: most_common_ndc[row['DRUG']] if pd.isna(row['NDC']) else row['NDC'], axis=1)

IV_PRESCRIPTIONS = IV_PRESCRIPTIONS.reset_index(drop=True)
IV_PRESCRIPTIONS = IV_PRESCRIPTIONS[~(IV_PRESCRIPTIONS.NDC=='0')]

IV_PRESCRIPTIONS = IV_PRESCRIPTIONS.drop_duplicates(keep='first')
IV_PRESCRIPTIONS = IV_PRESCRIPTIONS.reset_index(drop=True)
print(IV_PRESCRIPTIONS.shape,len(IV_PRESCRIPTIONS.HADM_ID.unique()),len(IV_PRESCRIPTIONS.ICUSTAY_ID.unique()))

In [ ]:
III_PRESCRIPTIONS.to_csv('ouput/III_M.csv',index=False)
IV_PRESCRIPTIONS.to_csv('ouput/IV_M.csv',index=False)